In [1]:
import random
import time

from montgomery_xz import KummerLine
from montgomery_isogeny import KummerLineIsogenyComposite
from utilities import torsion_basis


In [2]:
# Import pari interface
import cypari2

# SageMath imports
from sage.all import ZZ

# Make instance of Pari
pari = cypari2.Pari()

def discrete_log_pari(a, base, order):
    """
    Wrapper around pari discrete log.
    Works like a.log(b), but allows
    us to use the optional argument
    order.
    """
    x = pari.fflog(a, base, order)
    return ZZ(x)


In [3]:
def BiDLP(R, P, Q, D):
    """
    Given a basis P,Q of E[D] finds
    a,b such that R = [a]P + [b]Q.

    Uses the fact that 
        e([a]P + [b]Q, [c]P + [d]Q) = e(P,Q)^(ad-bc)
    """
    # e(P,Q)
    pair_PQ = P.weil_pairing(Q, D, algorithm="pari")

    # Write R = aP + bQ for unknown a,b
    # e(R, Q) = e(P, Q)^a
    pair_a = R.weil_pairing(Q, D, algorithm="pari")

    # e(R,-P) = e(P, Q)^b
    pair_b = R.weil_pairing(-P, D, algorithm="pari")

    # Now solve the dlog on Fq
    a = discrete_log_pari(pair_a, pair_PQ, D)
    b = discrete_log_pari(pair_b, pair_PQ, D)

    return a, b


In [5]:
def make_prime(p):
    '''
    Given a value `p`, finds a cofactor `f`
    such that p*f - 1 is prime.
    '''
    for i in range(1000):
        if (p*i - 1).is_prime():
            return p*i - 1, i

In [81]:
lam = 128 # 192, 256
ternary = False
params_binSIDH_t = [40, 60, 80] # t
params_binSIDH_t1= [20, 30, 40] # t1
params_terSIDH_t = [36, 56, 70] # t
params_terSIDH_t1 = [20, 25, 35] # t1

t = params_binSIDH_t[2]
t1 = params_binSIDH_t1[2]
sk_choice = [1, 2]

In [82]:
##### Setup ############################
A = prod(Primes()[1:2*t+1:2]) # The 2* ensures that p \equiv 3 mod 4
B = prod(Primes()[2:2*t+1:2])

a = prod(Primes()[1:2*t1+1:2])
b = prod(Primes()[2:2*t1+1:2])
p, f = make_prime(4*A*B)

FF.<x> = GF(p)[]
F.<i> = GF(p^2, modulus=x^2+1) 
E0 = EllipticCurve(F, [0, 6, 0, 1, 0])
E0.set_order((p+1)**2) 

PA, QA = torsion_basis(E0, A)
Pa, Qa = A//a*PA , A//a*QA
assert Pa.weil_pairing(Qa, a) != 1
Ra = Pa + Qa
PAa, QAa = PA*a , QA*a
assert PAa.weil_pairing(QAa, A//a) != 1
PB, QB = torsion_basis(E0, B)
Pb, Qb = B//b*PB , B//b*QB
assert Pb.weil_pairing(Qb, b) != 1
Rb = Pb + Qb
PBb, QBb = PB*b , QB*b
assert PBb.weil_pairing(QBb, B//b) != 1

In [83]:
if A//2 * PA == E0(0, 0):
    PA = PA + QA
elif A//2 * QA == E0(0, 0):
    QA = PA + QA 

assert A//2 * PA != E0(0, 0) and A//2 * QA != E0(0, 0)
E0 = KummerLine(E0)
xPA, xQA = E0(PA[0]), E0(QA[0])
xPa, xQa = E0(Pa[0]), E0(Qa[0])
xRa = E0(Ra[0])
xPAa, xQAa = E0(PAa[0]), E0(QAa[0])
xPB, xQB = E0(PB[0]), E0(QB[0])
xPb, xQb = E0(Pb[0]), E0(Qb[0])
xRb = E0(Rb[0])
xPBb, xQBb = E0(PBb[0]), E0(QBb[0])

In [13]:
def generate_secret_key(typ = ternary, Alice = True):
    
    # generate secret key
    if typ and Alice:
        m = 1
        n = random.randint(0, a-1)
    elif typ == False and Alice == False:
        m = 1
        n = random.randint(0, b-1)
    elif typ == True and Alice:
        m = random.randint(0, a-1)
        n = random.randint(0, a-1)
    else:
        m = random.randint(0, b-1)
        n = random.randint(0, a-1)
    #sk1 = [m,n]
    sk1 = [1, n]
    sk2 = random.choices(sk_choice, k =t-t1) 
    sk = (sk1, sk2)
    
    return sk

In [43]:
def compute_kernel_scalars(s, Alice = True):
    """ 
    Given a ternary secret `s`, returns scalars `B0` and `B1`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B0]*P + [B1]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B0]*P and [B1]*Q, which is used in the isogeny computations.
    """
    assert len(s) == t - t1
    B0 = 1
    B1 = 1
    order0 = 1
    order1 = 1

    #t = len(s)

    if Alice:
        P = Primes()[2*t1+1:2*t+1:2]
    else:
        P = Primes()[2*t1+2:2*t+1:2]

    for i, p in enumerate(P):
        if s[i] == 2:
            B1 *= p
            order0 *= p
        elif s[i] == 1:
            B0 *= p
            order1 *= p
        else:
            B0 *= p
            B1 *= p
    
    return B0, B1, order0, order1

In [44]:
compute_kernel_scalars([1,1,1,1],False)

(3524483, 1, 1, 3524483)

In [27]:
def compute_kernel(sk, Alice = True):
    """
    Compute the first kernel of isogeny
    [m]Pa + [n]Qa
    
    """
    m,n = sk[0]
    B0, B1, order0, order1 = compute_kernel_scalars(sk[1], Alice = Alice)
    if Alice:
    
        ker1 = m*Pa + n*Qa  #first kernel
        
        ker2 = B0*PAa + B1*QAa #second kernel
        
        ker1order = a//gcd(m,n)
        
    else:
        ker1 = m*Pb + n*Qb  #first kernel
        
        ker2 = B0*PBb + B1*QBb #second kernel
    
        ker1order = b//gcd(m,n)
    
    ker2order = order0*order1
    return ker1 ,ker2, ker1order, ker2order

In [28]:
def keygen(Alice = True):
    
    # genrate secret key
    sk = generate_secret_key(typ = ternary, Alice = Alice)
    print(sk)
    # compute kernel
    ker1, ker2, ker1order, ker2order = compute_kernel(sk, Alice)
    ker = ker1 + ker2
    xker = E0(ker[0])
    assert gcd(ker1order, ker2order) == 1
    phi = KummerLineIsogenyComposite(E0, xker, ker1order*ker2order)

    
    # Generate the masking values
    modulus = B if Alice else A # modulus is the order of torsion points
    mask = modulus
    #mask = 2
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus
    
    
    if Alice:
    # compute public key for alice (PB, QB, Rb)
        phixPB , phixQB = phi(xPB), phi(xQB)
        phixRb = phi(xRb)
        phixPB = mask *phixPB
        phixQB = mask_inv *phixQB
        E1 = phi.codomain()
        pk = (E1, phixPB, phixQB, phixRb)
    else:
    # compute public for bob (PA, QA, Ra)
        phixPA , phixQA = phi(xPA), phi(xQA)
        phixRa = phi(xRa)
        phixPA = mask *phixPA
        phixQA = mask_inv *phixQA
        E1 = phi.codomain()
        pk = (E1, phixPA, phixQA, phixRa)
    return phi, sk, pk

In [30]:
def compute_torsion_alice(m, n, P, Q, R):
    """
    Given image points P, Q, R
    """
    #Pa, Qa = A//a*PA, A//a*QA
    r, s = BiDLP(Ra, Pa, Qa, a) 
    Pa1, Qa1, Ra1 = A//a*P, A//a*Q, R
    r1, s1 = BiDLP(Ra1, Pa1, Qa1, a)
    #assert gcd(r,s) == 1
    #assert gcd(r1, s1) == 1
    #assert gcd(s1, a) == 1
    r = Zmod(a)(r)
    s = Zmod(a)(s)
    ker1 = m*(r1*s)*Pa1 + n*(s1*r)*Qa1
    return ker1

In [31]:
def compute_torsion_bob(m, n, P, Q, R):
    #Pb, Qb = B//b*PA, B//b*QA
    r, s = BiDLP(Rb, Pb, Qb, b) 
    Pb1, Qb1, Rb1 = B//b*P, B//b*Q, R
    r1, s1 = BiDLP(Rb1, Pb1, Qb1, b)
    print(r1, s1)
    #assert gcd(r,s) == 1
    #assert gcd(r1, s1) == 1
    #assert gcd(s1, b) == 1
    r = Zmod(b)(r)
    s = Zmod(b)(s)
    ker1 = m*(r1*s)*Pb1 + n*(s1*r)*Qb1
    return ker1

In [32]:
def shared_alice(sk, pk):
    m, n = sk[0]
    ska = sk[1]
    P, Q, R = pk[1].curve_point(), pk[2].curve_point(), pk[3].curve_point()
    ker1 = compute_torsion_alice(m, n, P, Q, R)
    k1, k2, o1, o2 = compute_kernel_scalars(ska, Alice = True)
    ker2 = k1*a*P + k2*a*Q
    ker = ker1 + ker2
    E1 = pk[0]
    ker = E1(ker)
    phi = KummerLineIsogenyComposite(E1, ker, o1*o2*a)
    EAB = phi.codomain()
    return EAB.j_invariant()

In [63]:
def shared_bob(sk, pk):
    m, n = sk[0]
    skb = sk[1]
    P, Q, R = pk[1].curve_point(), pk[2].curve_point(), pk[3].curve_point()
    ker1 = compute_torsion_bob(m, n, P, Q, R)
    k1, k2, o1, o2 = compute_kernel_scalars(skb, Alice = False)
    ker2 = k1*b*P + k2*b*Q
    ker = ker1 + ker2
    E1 = pk[0]
    ker = E1(ker)
    phi = KummerLineIsogenyComposite(E1, ker, o1*o2*b)
    EBA = phi.codomain()
    return EBA.j_invariant()

In [50]:
phiA, ska, pka = keygen(Alice = True)
phiB, skb, pkb = keygen(Alice = False)

([1, 83], [2, 1, 1, 1])
([1, 7304], [2, 1, 2, 2])


In [51]:
EEA, UB, VB, Xb = pka[0], pka[1].curve_point(), pka[2].curve_point(), pka[3].curve_point()
EEB, UA, VA, Xa = pkb[0], pkb[1].curve_point(), pkb[2].curve_point(), pkb[3].curve_point()

In [52]:
Ba0, Ba1, ordera0, ordera1 = compute_kernel_scalars(ska[1], Alice = True)
Bb0, Bb1, orderb0, orderb1 = compute_kernel_scalars(skb[1], Alice = False)

In [53]:
kera1 = Pa + ska[0][1]*Qa
kera2 = Ba0*PAa + Ba1*QAa 
kera = kera1 + kera2

In [54]:
kerb1 = Pb + skb[0][1]*Qb
kerb2 = Bb0*PBb + Bb1*QBb
kerb = kerb1 + kerb2

In [56]:
publicA = KummerLineIsogenyComposite(E0, E0(kera), A).codomain()
publicB = KummerLineIsogenyComposite(E0, E0(kerb), B).codomain()

In [57]:
EEA.j_invariant()

1757966576102461675051*i + 65099101396921486896

In [58]:
publicA.j_invariant()

1757966576102461675051*i + 65099101396921486896

In [59]:
EEB.j_invariant()

22044530977804595536958*i + 1928426725319153100666

In [60]:
publicB.j_invariant()

22044530977804595536958*i + 1928426725319153100666

In [61]:
shared_alice(ska, pkb)

16187274261346524469629*i + 22050555126208553457050

In [64]:
shared_bob(skb, pka)

10843 3802


16187274261346524469629*i + 22050555126208553457050

In [65]:
Ker = kera + kerb

In [66]:
sharedkey = KummerLineIsogenyComposite(E0, E0(Ker), A*B).codomain()

In [67]:
sharedkey.j_invariant()

16187274261346524469629*i + 22050555126208553457050

In [84]:
N = 5
tt = [0, 0, 0, 0]

for _ in range(N):
    t0 = time.process_time_ns()
    phiA, skA, pkA = keygen(Alice=True)
    tt[0] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    phiB, skB, pkB = keygen( Alice=False)
    tt[1] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    ssA = shared_alice(skA, pkB)
    tt[2] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    ssB = shared_bob(skB, pkA)
    tt[3] += time.process_time_ns() - t0

    assert ssA == ssB

tt = [float(t) / N / 10^6 for t in tt]

print(f"KeyGen_A took {(tt[0]):.1f} ms")
print(f"KeyGen_B took {(tt[1]):.1f} ms")
print(f"shared_A took {(tt[2]):.1f} ms")
print(f"shared_B took {(tt[3]):.1f} ms")

([1, 123123419433811032727298622465946002040577614141130342132099927062759622504124478424], [1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1])
([1, 1453701975693390413450898691972735984662445462040865851410888669412480038890499083092], [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2])
424531539997423599586205636125893658240290663522034631075967606482050203133604227119 4423953011866800912057079155349240374474308122052062231725500736568126117226483983474
([1, 407444487313594300470335200575306766281050414856207462144184852713721064222608669194], [2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1])
([1, 6604295064766231096619448744590407233046672489873024345099966387443410907334483697333], [2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2,